# Brilliant Diamonds – Datenaufbereitung

In [ ]:
import pandas as pd
import random

Daten einlesen und prüfen

In [ ]:
dia = pd.read_csv('diamonds_dataset.csv')
dia.info()

Erste Zeilen anzeigen

In [ ]:
dia.head()

Nichtbenötigte Spalten entfernen.

In [ ]:
diaDrop = dia.drop(['id', 'url', 'date_fetched'], axis=1)
diaDrop.head(10)

Einfache Statistik der metrischen Features

In [ ]:
diaDrop.describe()

Einfache Statistik der kategorialen Features.

In [ ]:
diaDrop.describe(include='object')

Zählen aller Kategorien

In [ ]:
for feature in ('shape', 'cut', 'color', 'clarity', 'report', 'type'):
    print(feature)
    print(diaDrop[feature].value_counts())

### Bedeutung der Kategorien

Clarity: https://en.wikipedia.org/wiki/Diamond_clarity:
 
(best) FL IF VVS1 VVS2 VS1 VS2 SI1 SI2 I1 I2 I3 (worse)
 
Color: https://en.wikipedia.org/wiki/Diamond_color
 
(best) D ... J (worse)

Cut: ebenfalls ordinal, Namen sind sprechend

Shape, Report, Typ: kategorial, keine Ordnung

Zuordnen von Zahlen zu jeder Kategorie gemäßig Index der jeweiligen Kategorieliste.

In [ ]:
def getMap(values):
    res = dict()
    for i, key in enumerate(values):
        res[key] = i
    return res

In [ ]:
lColor = [x for x in 'JIHGFED']
lCuts = ['Fair', 'Good', 'Very Good', 'Ideal', 'Super Ideal']
lClarities = ['SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF', 'FL']
lShapes = diaDrop['shape'].unique()
lReports = diaDrop['report'].unique()
lTypes = diaDrop['type'].unique()
mCol = getMap(lColor)
mCut = getMap(lCuts)
mClar = getMap(lClarities)
mShape = getMap(lShapes)
mRep = getMap(lReports)
mTyp = getMap(lTypes)

Mappen der Zahlen auf die Kategorien.

In [ ]:
diaDrop['icol'] = diaDrop.color.map(mCol)
diaDrop['icut'] = diaDrop.cut.map(mCut)
diaDrop['iclar'] = diaDrop.clarity.map(mClar)
diaDrop['ishape'] = diaDrop['shape'].map(mShape)
diaDrop['irep'] = diaDrop.report.map(mRep)
diaDrop['ityp'] = diaDrop.type.map(mTyp)
diaDrop.head(10)

Prüfen auf fehlende Werte (vergessene Kategorien?).

In [ ]:
diaDrop[diaDrop.isna().any(axis=1)]

Verwerfen der ursprünglichen Kategoriespalten

In [ ]:
diaNum = diaDrop.drop(['shape', 'cut', 'color', 'clarity', 'report', 'type'], axis=1)
diaNum.head()

Nochmal die Statistik. Die Aussagekraft für die Kategoriespalten ist eher gering.

In [ ]:
diaNum.describe()

Prüfen der Dimensionen.

In [ ]:
diaNum.shape

Aufteilen der Daten in 5000 Trainingsdaten und den Rest als Testdaten. Vorher durchmischen.

In [ ]:
diaCounts = diaNum.shape[0]
shuffleIdx = [x for x in range(diaCounts)]
random.shuffle(shuffleIdx)
shuffleIdx[:50]
idxTrain = shuffleIdx[:5000]
idxTest = shuffleIdx[5000:]
diaTrain = diaNum.iloc[idxTrain, :]
diaTest = diaNum.iloc[idxTest, :]
print(diaTrain.shape, diaTest.shape)

Speichern der Trainings- und Testdaten für die spätere Nutzung.

In [ ]:
diaTrain.to_csv('brilliantdia-train.csv', index=False)
diaTest.to_csv('brilliantdia-test.csv', index=False)